<a href="https://colab.research.google.com/github/mgersins-design/Machine-Learning/blob/main/HW7/ResNet_10_with_Dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

import datetime

In [2]:
transform_process = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2470, 0.2435, 0.2616]
    )
])

data_path = './data'

cifar10_train = datasets.CIFAR10(
    data_path, train=True, download=True, transform=transform_process
)
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True, transform=transform_process
)

batch_size = 256
train_loader = DataLoader(cifar10_train, batch_size=batch_size, shuffle=True, num_workers = 2)
val_loader = DataLoader(cifar10_val, batch_size=batch_size, shuffle=False, num_workers = 2)

print(f"Trainingsdaten: {len(cifar10_train)} Bilder")
print(f"Validierungsdaten: {len(cifar10_val)} Bilder")

100%|██████████| 170M/170M [00:13<00:00, 12.2MB/s]


Trainingsdaten: 50000 Bilder
Validierungsdaten: 10000 Bilder


In [3]:
class ResBlock (nn.Module):
  def __init__(self, n_chans):
    super(ResBlock, self).__init__()
    self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3, padding=1, bias=False)
    self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
    torch.nn.init.kaiming_normal_(self.conv.weight, nonlinearity='relu')
    torch.nn.init.constant_(self.batch_norm.weight, 0.5)
    torch.nn.init.zeros_(self.batch_norm.bias)

  def forward(self, x):
    out = self.conv(x)
    out = self.batch_norm(out)
    return torch.relu(out + x)

In [4]:
class NetResDeep (nn.Module):
  def __init__(self, n_chans1=32, n_blocks=10):
    super().__init__()
    self.n_chans1 = n_chans1
    self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
    #self.resblocks = nn.Sequential(*(n_blocks * [ResBlock(n_chans=n_chans1)]))
    self.resblocks = nn.Sequential(*[ResBlock(n_chans=n_chans1) for _ in range(n_blocks)])
    #self.conv1_dropout = nn.Dropout2d(p=0.3)
    self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
    self.dropout = nn.Dropout(p=0.3)
    self.fc2 = nn.Linear(32, 10)

  def forward(self, x):
    out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
    out = self.resblocks(out)
    #out = self.conv1_dropout(out)
    out = F.max_pool2d(out, 2)
    out = out.view(-1, 8*8*self.n_chans1)
    out = torch.relu(self.fc1(out))
    out = self.dropout(out)
    out = self.fc2(out)
    return out

In [5]:
# Switching to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {device}")

Training on cuda


In [6]:
def train_and_validate(n_epochs, optimizer, model, loss_fn, train_loader, val_loader):
  device = next(model.parameters()).device

  for epoch in range(1, n_epochs+1):
    # Training Phase
    model.train()
    loss_train = 0.0
    correct_train = 0
    total_train = 0

    for imgs, labels in train_loader:
      imgs = imgs.to(device = device)
      labels = labels.to(device = device)

      outputs = model(imgs)
      loss = loss_fn(outputs, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      loss_train += loss.item()

    model.eval()
    correct_val = 0
    total_val = 0

    with torch.no_grad():
      for imgs, labels in val_loader:
        imgs = imgs.to(device = device)
        labels = labels.to(device = device)

        outputs = model(imgs)
        _, predicted = torch.max(outputs, dim=1)
        total_val += labels.shape[0]
        correct_val += int((predicted == labels).sum())

    avg_loss = loss_train / len(train_loader)
    acc_val = correct_val / total_val

    if epoch == 1 or epoch % 10 == 0:
      print('{} Epoch {:3d}, Training loss {:.4f}, Val Acc: {:.2f}'.format(
          datetime.datetime.now(), epoch, avg_loss, acc_val))

In [7]:
model = NetResDeep(n_chans1=32, n_blocks=10).to(device=device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
loss_fn = nn.CrossEntropyLoss()
n_epochs = 100

train_and_validate(n_epochs=n_epochs, optimizer=optimizer, model=model, loss_fn=loss_fn, train_loader=train_loader, val_loader=val_loader)

2025-11-26 05:37:17.511877 Epoch   1, Training loss 1.9019, Val Acc: 0.41
2025-11-26 05:38:22.227518 Epoch  10, Training loss 1.2690, Val Acc: 0.65
2025-11-26 05:39:34.237600 Epoch  20, Training loss 1.0871, Val Acc: 0.71
2025-11-26 05:40:46.277068 Epoch  30, Training loss 0.9785, Val Acc: 0.72
2025-11-26 05:41:57.807717 Epoch  40, Training loss 0.9035, Val Acc: 0.71
2025-11-26 05:43:10.844175 Epoch  50, Training loss 0.8500, Val Acc: 0.70
2025-11-26 05:44:22.716720 Epoch  60, Training loss 0.8129, Val Acc: 0.72
2025-11-26 05:45:35.497776 Epoch  70, Training loss 0.7683, Val Acc: 0.73
2025-11-26 05:46:47.203779 Epoch  80, Training loss 0.7521, Val Acc: 0.72
2025-11-26 05:48:00.759049 Epoch  90, Training loss 0.7327, Val Acc: 0.72
2025-11-26 05:49:13.331430 Epoch 100, Training loss 0.7064, Val Acc: 0.73


In [8]:
# Model Complexity
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(159594,
 [864,
  32,
  9216,
  32,
  32,
  9216,
  32,
  32,
  9216,
  32,
  32,
  9216,
  32,
  32,
  9216,
  32,
  32,
  9216,
  32,
  32,
  9216,
  32,
  32,
  9216,
  32,
  32,
  9216,
  32,
  32,
  9216,
  32,
  32,
  65536,
  32,
  320,
  10])